# LLaMA-3 모델 호출하기

In [2]:
model_id = "allganize/Llama-3-Alpha-Ko-8B-Instruct"

In [4]:
import torch
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_id).to("cpu")

quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

device = torch.device("mps" if torch.has_mps else "cpu")
quantized_model.to(device)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

: 

# LLaMA-3 Tokenizer
LLaMA는 페이스북에서 2023년 2월에 출시한 공개 LLM 모델입니다. 7B(70억) ~ 65B(650억)에 이르는 다양한 크기의 모델을 허깅페이스에 공개하여 사용할 수 있도록 하였습니다.

공개된 모델이기 때문에 허깅페이스에서 다운로드 받아 사용할 수 있습니다.

* local model 사용 시에는 Huggingface 에 제공되는 config.json 을 확인하자

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_id)

## Encode & Decode

In [7]:
print(tokenizer.encode("오늘은 배가 너무 고프다"))

[128000, 58368, 105622, 34804, 74769, 20565, 109748, 46230, 58260, 63644, 13447]


In [8]:
# LLaMA 에서는 문장 앞에 <begin_of_text> 가 붙은 채로 훈련이 된다
print(
    tokenizer.decode(
        [128000, 58368, 105622, 34804, 74769, 20565, 109748, 46230, 58260, 63644, 13447]
    )
)

<|begin_of_text|>오늘은 배가 너무 고프다


# Template
* 모델이 훈련될 때 입력으로 제공되었던 형식
* 모델마다 모두 다르게 설정되어 있다
* **학습 시 해당 template 을 따르지 않으면, 제대로 된 성능을 기대할 수 없다.**

## Role
최근 개발되는 LLM에는 역할(`role`)이라는 개념이 존재합니다. role은 크게 user, system, assistant가 존재합니다.
- `user` : 사용자 신분
  - 사용자가 챗봇에게 질문할 내용이 됩니다.
- `system` : LLM에게 시키고 싶은 것
  - 대답을 어떻게 생성할지 결정
  - LLM이 어떤 기준을 가지고 작동할지에 대한 설정이 가능합니다.
- `assistant` : LLM의 답변

개발자가 직접적으로 template을 신경쓰지 않도록 허깅페이스의 모델은 `apply_chat_template`이라는 함수를 통해 모델의 Chat Template을 자동으로 생성할 수 있게 도와줍니다. 때에 따라 이 함수를 사용하지 않고 개발하는 경우도 종종 있습니다.

In [9]:
messages = [
    {
        "role": "system",
        "content": "당신은 인공지능 비서입니다. 묻는 말에 친절하고 정확하게 답변하세요.",
    },
    {"role": "user", "content": "은행의 기준 금리에 대해서 설명해주세요"},
    # assistant 는 자동으로 생성
]

## ChatTemplate

In [10]:
# Tokenizer 를 이용해 LLaMA 에 맞는 ChatTemplate 만들기
# tokenize=True : template 자체가 정수 인코딩된다
template_messages = tokenizer.apply_chat_template(messages, tokenize=True)
template_messages

[128000,
 128006,
 9125,
 128007,
 271,
 65895,
 83628,
 34804,
 59777,
 103896,
 67119,
 75086,
 27796,
 80052,
 13,
 100570,
 119,
 16969,
 101264,
 19954,
 108280,
 104834,
 101360,
 127923,
 102893,
 111964,
 92245,
 13,
 128009,
 128006,
 882,
 128007,
 271,
 34804,
 101066,
 21028,
 111902,
 104193,
 111833,
 122115,
 114942,
 34983,
 92769,
 128009]

## `assistant` 적용하기
`assistant` 토큰은 LLM 모델이 여기부터 답변을 하는 구간이다 라는 것을 학습하기 위한 토큰입니다. 따라서 LLM이 학습될 때 다음과 같이 훈련 되었다는 이야기 입니다.
```
<system> 당신은 인공지능 비서입니다 </system>
<user> 손흥민의 소속 팀은? </user>
<assistant> 손흥민의 소속 팀은 토트넘 ~~~ </assistant>
```

따라서 LLM을 이용해 챗봇에 대한 답변을 받기 위해서는 다음과 같이 `assistant`를 추가해서 물어봐야 합니다.
```
<system> 당신은 인공지능 비서입니다 </system>
<user> 손흥민의 소속 팀은? </user>
<assistant>
```

만약 `assistant`를 삽입하지 않고 챗봇에게 요청하면 오동작할 가능성이 생기게 됩니다.
```
<system> 당신은 인공지능 비서입니다 </system>
<user> 손흥민의 소속 팀은? </user>
```

하지만 매번 LLM의 Template에 맞춰서 `assistant`를 넣어주는 것은 매우 번거로운 작업이기 때문에 `add_generation_prompt` 옵션을 이용하여 자동으로 `assistant`를 추가하게 할 수 있습니다.

In [11]:
# add_generation_prompt=False 로 적용
template_messages = tokenizer.apply_chat_template(
    conversation=messages, add_generation_prompt=False, tokenize=False
)

print(template_messages)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 인공지능 비서입니다. 묻는 말에 친절하고 정확하게 답변하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

은행의 기준 금리에 대해서 설명해주세요<|eot_id|>


In [12]:
# add_generation_prompt=True 로 적용
template_messages = tokenizer.apply_chat_template(
    conversation=messages, add_generation_prompt=True, tokenize=False
)

print(template_messages)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 인공지능 비서입니다. 묻는 말에 친절하고 정확하게 답변하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

은행의 기준 금리에 대해서 설명해주세요<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [13]:
# LLM에는 텍스트 생성이 종료 되었다는 의미의 stop_token이 존재합니다.
# 모델이 글을 쓰다가 멈추는 시점을 결정하는 토큰입니다.
# 모든 모델에 다 있는 것은 아닙니다.
tokenizer.eos_token_id

128001

In [14]:
tokenizer.decode(tokenizer.eos_token_id)

'<|end_of_text|>'

In [15]:
messages = [
    {
        "role": "system",
        "content": "당신은 챗봇입니다. 묻는 말에 친절하고 정확하게 답변해주세요",
    },
    {"role": "user", "content": "당신이 누군지 설명해주세요"},
]

In [16]:
encodes = tokenizer.apply_chat_template(
    conversation=messages,
    add_generation_prompt=True,
    return_tensors="pt",  # PyTorch tensor 반환
)

encodes

tensor([[128000, 128006,   9125, 128007,    271,  65895,  83628,  34804, 102783,
            245, 103850,    229,  80052,     13, 100570,    119,  16969, 101264,
          19954, 108280, 104834, 101360, 127923, 102893, 111964,  34983,  92769,
         128009, 128006,    882, 128007,    271,  65895,  83628,  13094, 106200,
         102323,  22035, 114942,  34983,  92769, 128009, 128006,  78191, 128007,
            271]])

In [17]:
model_inputs = encodes.to(device)
model_inputs

tensor([[128000, 128006,   9125, 128007,    271,  65895,  83628,  34804, 102783,
            245, 103850,    229,  80052,     13, 100570,    119,  16969, 101264,
          19954, 108280, 104834, 101360, 127923, 102893, 111964,  34983,  92769,
         128009, 128006,    882, 128007,    271,  65895,  83628,  13094, 106200,
         102323,  22035, 114942,  34983,  92769, 128009, 128006,  78191, 128007,
            271]], device='mps:0')

**시스템 프롬프트**
```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 챗봇 한경이 입니다. 묻는 말에 친절하고 정확하게 답변하세요<|eot_id|>
```

**유저 프롬프트**
```
<|start_header_id|>user<|end_header_id|>

당신이 누군지 설명해주십시오<|eot_id|>
```

**어시스턴스 시작점**
```
<|start_header_id|>assistant<|end_header_id|>


```

**답변 생성하기**
LLM에서 답변을 받기 위해서는 generate 함수를 사용

In [18]:
generated_ids = model.generate(
    model_inputs,  # 정수 인코딩이 된 프롬프트(Chat Template)
    max_new_tokens=512,  # 모델이 대답할 응답 메시지의 최대 길이
    eos_token_id=tokenizer.eos_token_id,  # 모델의 텍스트 생성 종료 토큰
    repetition_penalty=1.05,  # 반복된 응답 지수. 값이 작아질 수록 반복
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/opt/anaconda3/envs/dl-env/lib/python3.11/site-packages/transformers/generation/utils.py:1934: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on mps, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generate()`.
  warnings.warn(


RuntimeError: Placeholder storage has not been allocated on MPS device!

In [11]:
# batch_decode : 여러 개의 input에 대해 한꺼번에 변환
decoded = tokenizer.batch_decode(generated_ids)
decoded

NameError: name 'generated_ids' is not defined